In [1]:
def add_zero(s):

    if len(s) < 2:

        return "0" + s

    else:

        return s

In [2]:
import urllib
def downloadNDays(y, m, d, duration):
    for i in range(duration):
        Y= add_zero(str(y))
        M= add_zero(str(m))
        D= add_zero(str(d))
        for i in range(24):
            urllib.urlretrieve(("http://data.githubarchive.org/%s-%s-%s-%s.json.gz" %(Y, M, D, i)), ("/tmp/%s-%s-%s-%s.json.gz" %(Y, M, D, i)))
        d= d+1
## define the download functions to get data from github archive

In [3]:
downloadNDays(2017,3,2,1)
## download the data from 2017-3-1 to 2017-3-3

In [4]:
%fs
ls file:/tmp/

In [5]:
PATH_PREFIX= "/tmp/"
EXT_JG=".json.gz"
EXT_JSON= ".json"

def get_file_name_json_gz(para_list):

    return get_file_name(para_list) + EXT_JG

def get_file_name_json(para_list):

    return get_file_name(para_list) + EXT_JSON

def get_file_name(para_list):

    list_ = list(map(str,para_list))

    list_[1] = add_zero(list_[1])    

    list_[2] = add_zero(list_[2])

    return '-'.join(list_)

    


In [6]:
import gzip
def merge_one_day(cy, cm, cd):

   tmp = []

   for hh in range(24):

       with gzip.open(PATH_PREFIX + get_file_name_json_gz([cy, cm, cd, hh]), 'r') as f_in:

           tmp += f_in


   with open(PATH_PREFIX + get_file_name_json([cy, cm, cd, 0]), 'wb') as f_out:

       for line in tmp:

           f_out.write(line)



In [7]:
merge_one_day(2017,3,2)
## decompress the data and put these json files into one json

In [8]:
import shutil
with open('/tmp/2017-03-01-0.json', 'rb') as f_in:
  with gzip.open('/tmp/2017-03-01.json.gz', 'w') as f_out:
    shutil.copyfileobj(f_in,f_out)
    

In [9]:

dbutils.fs.mv("file:/tmp/2017-03-01-0.json", "dbfs:/FileStore/tables/gh_archive/2017-01-05-0.json")

In [10]:
%fs
ls file:/tmp/

In [11]:
from pyspark.sql.types import *

inputPath = "file:/tmp/2017-03-02-0.json"


InputDF = sqlContext.read.json(inputPath)

InputDF.show()
# read the json file into a spark dataframe

In [12]:
from pyspark.sql import functions
githubInput = InputDF.withColumn("actorID", InputDF.actor.id)
githubInput = githubInput.withColumn("actorName", InputDF.actor.login)
githubInput = githubInput.withColumn("repoId", InputDF.repo.id)
githubInput = githubInput.withColumn("repoName", InputDF.repo.Name)

In [13]:
githubInputDF= githubInput.select("id","type","actorID","actorName","repoID","repoName","created_at")

In [14]:
from pyspark.sql.functions import count
githubGroupDF= githubInputDF.groupBy(['actorID','actorName','type']).agg(count("id").alias("_count"))

In [15]:
githubGroupDF.take(5)

In [16]:
typeList= githubGroupDF.select('type').distinct().rdd.map(lambda r: r[0]).collect() ## without actorID

In [17]:
typeList2= [str(x) for x in typeList]

In [18]:
type_list= []
type_list.extend(['actorID'])
type_list.extend(typeList2)


In [19]:
type_list ## with actorID

In [20]:
githubGroupedData= githubGroupDF.groupBy(['actorID','actorName']).pivot('type', typeList ).sum("_count")
## get the number of events for each user in 2017-3-1

In [21]:
githubGroupedData.show()

In [22]:
githubRdd = githubGroupedData.select(typeList).filter("PushEvent<100").na.fill(0).rdd

In [23]:
from pyspark.mllib.stat import Statistics
summary = Statistics.colStats(githubRdd.map(list))
print(summary.mean())  # a dense vector containing the mean value for each column
print(summary.variance())  # column-wise variance
print(summary.numNonzeros())  # number of nonzeros in each column
# get the statistics of the data

In [24]:
githubRdd2= githubGroupedData.select(type_list).filter("PushEvent<100").na.fill(0).rdd

In [25]:
githubRdd2.toDF().createOrReplaceTempView('github')

In [26]:
%sql
select * from github

In [27]:
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
clusters = KMeans.train(githubRdd.map(list), 3, maxIterations=15, initializationMode="k-means||" )

# feed the data into the kmeans model

In [28]:
clusters.centers

In [29]:
clusterLabel= clusters.predict(githubRdd.map(list))
## get the cluster of each user

In [30]:
index= sc.parallelize(clusterLabel.collect())
z= index.zip(sc.parallelize(githubRdd2.map(list).collect()))

In [31]:
z.take(5)

In [32]:
new_rdd= z.map(lambda (x,y): [x, y[0], y[1], y[2], y[3], y[4], y[5], y[6], y[7], y[8], y[9], y[10], y[11], y[12], y[13], y[14]])

In [33]:
type_list2= []
type_list2.extend(['clusters'])
type_list2.extend(type_list)
new_df= new_rdd.toDF(type_list2)

In [34]:
new_df.show()

In [35]:
new_df.createOrReplaceTempView('githubCluster')

In [36]:
%sql
select clusters, avg(PushEvent) push, avg(GollumEvent) Gollum, avg(ReleaseEvent) Release, avg(CommitCommentEvent) Commit, avg(CreateEvent) create_, avg(PullRequestReviewCommentEvent) PullRequestComment, avg(IssueCommentEvent) IssueComment, avg(DeleteEvent) Delete, avg(IssuesEvent) Issue, avg(ForkEvent) Fork, avg(PublicEvent) Public, avg(MemberEvent) Member, avg(WatchEvent) Watch, avg(PullRequestEvent) PullRequest from githubCluster
group by 
clusters
/* plot the average number of actions for each clusters */

In [37]:
from math import sqrt

def error(point): 
    center = clusters.centers[clusters.predict(point)] 
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = (githubRdd.map(lambda point:error(point)).reduce(lambda x, y: x+y))
print('Within Set Sum of Squared Error = ' + str(WSSSE))
# define the cost function for each kmeans model.